In [1]:
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [87]:
# ФРАНЦУЗСКИЕ ТОПИКИ

dir = 'fr_corpus_POS_tagged'
filepaths = []

for i in os.listdir(dir):
    filepaths.append(os.path.join(dir, i))  

fr_corpus = []
fr_tokens = []

for filepath in filepaths:
    with open(filepath, encoding='utf-8') as f:
        dialogues = f.read().split('\n')
        for dialogue in dialogues:
            dialogue = dialogue.split()
            dialogue_nouns = []
            for word in dialogue:
                word = word.split('_')
                if word[1] == 'NOUN':
                    dialogue_nouns.append(word[0])
            if dialogue_nouns:
                fr_tokens.extend(dialogue_nouns)
                dialogue_nouns = ' '.join(dialogue_nouns)
                fr_corpus.append(dialogue_nouns)

fr_lemmata = set(fr_tokens)
print(f'Количество токенов: {len(fr_tokens)}')
print(f'Количество лемм: {len(fr_lemmata)}')
print(f'Количество документов в корпусе: {len(fr_corpus)}')

Количество токенов: 2397
Количество лемм: 1042
Количество документов в корпусе: 832


In [103]:
# РУССКИЕ ТОПИКИ

dir = 'rus_corpus_POS_tagged'
filepaths = []

for i in os.listdir(dir):
    filepaths.append(os.path.join(dir, i))  

rus_corpus = []
rus_tokens = []

for filepath in filepaths:
    with open(filepath, encoding='utf-8') as f:
        dialogues = f.read().split('\n')
        for dialogue in dialogues:
            dialogue = dialogue.split()
            dialogue_nouns = []
            for word in dialogue:
                word = word.split('_')
                if word[1] == 'NOUN' or word[1] == 'ADJF':
                    dialogue_nouns.append(word[0])
            if dialogue_nouns:
                rus_tokens.extend(dialogue_nouns)
                dialogue_nouns = ' '.join(dialogue_nouns)
                rus_corpus.append(dialogue_nouns)

rus_lemmata = set(rus_tokens)
print(f'Количество токенов: {len(rus_tokens)}')
print(f'Количество лемм: {len(rus_lemmata)}')
print(f'Количество документов в корпусе: {len(rus_corpus)}')

Количество токенов: 35667
Количество лемм: 4814
Количество документов в корпусе: 5746


In [38]:
corpus = fr_corpus + rus_corpus
tokens = fr_tokens + rus_tokens
lemmata = set.union(fr_lemmata, rus_lemmata)

print(f'Количество токенов: {len(tokens)}')
print(f'Количество лемм: {len(lemmata)}')
print(f'Количество документов в корпусе: {len(corpus)}')

Количество токенов: 122219
Количество лемм: 10107
Количество документов в корпусе: 7469


In [91]:
import random

print(fr_corpus)
for i in range(100):
    pass
    print(random.choice(fr_tokens))

['ангел', 'господин посольство', 'женщина петербург', 'паж', 'чулок', 'моя жена', 'ужин', 'желудок маша желудок', 'прелесть', 'ум', 'кружка', 'мужчина', 'удовольствие', 'натали', 'предмет ухаживание', 'страна', 'бог бог', 'брат сестра близость любовь дело', 'брак небо', 'тон', 'матушка', 'сын', 'сила приказ', 'жёлчь прилив голов', 'герцог несчастие характер спокойствие', 'тысяча войско слог', 'друг', 'дерево мрак меланхолия', 'смерть смерть страдание убежище', 'улыбка меланхолия', 'луч свет оттенок печаль отчаяние возможность утешение', 'пища душа счастие меланхолия мука уединение сладость слеза течение', 'жюли', 'источник князь василий сын жюли', 'наташа волос', 'персиянин долох', 'мила', 'неправда дюпора', 'женщина', 'графиня залог цветок', 'девочка', 'девочка', 'танец шаль', 'прелесть', 'газа', 'ум ум', 'прелесть', 'причина невеста жених свет скука', 'барышня', 'слово бог', 'натали слово', 'самонадеянность', 'княжна', 'ножка друг взгляд богиня', 'дружок', 'человек', 'след господин',

In [104]:
with open('ru_stop_words.txt', encoding='utf-8') as f:
    STOPWORDS = f.read().split()

no_features = 4500
no_topics = 10

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9,
    # min_df=3, 
    stop_words=STOPWORDS, 
    max_features=no_features
)
data_tfidf_vectorized = tfidf_vectorizer.fit_transform(rus_corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


nmf = NMF(n_components=no_topics, 
          random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd'
          )
nmf_tfidf_model = nmf.fit(data_tfidf_vectorized)
print(nmf_tfidf_model.transform(data_tfidf_vectorized).shape)

(5746, 10)


In [77]:
def print_topics(model, feature_names, top_n=10):
    for topic_idx, topic in enumerate(model.components_):
            print("Topic %d:" % (topic_idx))
            print(" ".join([feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]]))

In [105]:
print_topics(nmf_tfidf_model, tfidf_feature_names)


Topic 0:
пьер глаз офицер лицо солдат друг взгляд другой слово время
Topic 1:
князь андрей василий николай рука отец глаз сын старый кутузов
Topic 2:
ростов солдат глаз рука долох ответ комната деньга борис письмо
Topic 3:
наташа глаз графиня рука лицо николай комната другой слеза день
Topic 4:
княжна марья графиня дмитрий рука комната отец николай лицо другой
Topic 5:
ваш сиятельство благородие превосходительство величество светлость имя генерал дом высокопревосходительство
Topic 6:
граф старый сын кирилл минута время голова владимир раз безухов
Topic 7:
бог слава место графиня дом левый император николай государь слово
Topic 8:
денисов васька офицер петя рука графиня какой шельма лошадь досада
Topic 9:
соня слово николай борис голова мама гостиный николинька слеза лицо
